# 2. Classification

In [1]:
# Run this code to make Jupyter print every
# printable statement and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# To visualize the data
import matplotlib.pyplot as plt

# Generic libraries
import seaborn as sns
import pandas as pd
import numpy as np

# Regression models
import sklearn
import scipy
from scipy.stats import *
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV #split the data into training and test
from sklearn.linear_model import LinearRegression #linear regression
from sklearn.preprocessing import PolynomialFeatures #for polynomial regression
from sklearn.metrics import r2_score, mean_squared_error

# 5-folds crossvalidation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, classification_report, roc_auc_score
from sklearn.pipeline import Pipeline

#classification
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

## Preprocessing

In [10]:
training_set = pd.read_csv("training_set_complete.csv")
test_set = pd.read_csv("test_set_complete.csv")
#training_set = training_set.drop(columns=['Unnamed: 0'])
#test_set = test_set.drop(columns=['Unnamed: 0'])
training_set['Revenue'] = training_set['Revenue'].astype(int)
test_set['Revenue'] = test_set['Revenue'].astype(int)
#training_set['Revenue'].value_counts()
#test_set['Revenue'].value_counts()
categorical_features = ['Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jul', 'Month_June',
       'Month_Mar', 'Month_May', 'Month_Nov', 'Month_Oct', 'Month_Sep',
       'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3',
       'OperatingSystems_4', 'OperatingSystems_5', 'OperatingSystems_6',
       'OperatingSystems_7', 'OperatingSystems_8', 'Browser_1', 'Browser_2',
       'Browser_3', 'Browser_4', 'Browser_5', 'Browser_6', 'Browser_7',
       'Browser_8', 'Browser_9', 'Browser_10', 'Browser_11', 'Browser_12',
       'Browser_13', 'Region_1', 'Region_2', 'Region_3', 'Region_4',
       'Region_5', 'Region_6', 'Region_7', 'Region_8', 'Region_9',
       'TrafficType_1', 'TrafficType_2', 'TrafficType_3', 'TrafficType_4',
       'TrafficType_5', 'TrafficType_6', 'TrafficType_7', 'TrafficType_8',
       'TrafficType_9', 'TrafficType_10', 'TrafficType_11', 'TrafficType_12',
       'TrafficType_13', 'TrafficType_14', 'TrafficType_15', 'TrafficType_16',
       'TrafficType_17', 'TrafficType_18', 'TrafficType_19', 'TrafficType_20',
       'VisitorType_New_Visitor', 'VisitorType_Other',
       'VisitorType_Returning_Visitor', 'Weekend_False', 'Weekend_True']
#training_set[categorical_features] = training_set[categorical_features].astype('category')
#test_set[categorical_features] = test_set[categorical_features].astype('category')

In [11]:
X_train_full = training_set.drop(columns=['Revenue'])
X_test_full = test_set.drop(columns=['Revenue'])
y_train = training_set['Revenue']
y_test = test_set['Revenue']

### Variance selection
We apply variance selection to remove all the features with a very low variance.

In [12]:
# FACCIAMO QUESTO STEP PER EVITARE PROBLEMI DI CONVERGENZA NEL MODELLO
from sklearn.preprocessing import StandardScaler

# Assuming you have your feature data X
numeric_features = ['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']
categorical_features = ['Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jul', 'Month_June', 'Month_Mar', 'Month_May', 
                        'Month_Nov', 'Month_Oct', 'Month_Sep', 'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3', 'OperatingSystems_4', 'OperatingSystems_6', 'OperatingSystems_8', 'Browser_1', 'Browser_2', 'Browser_3', 'Browser_4', 'Browser_5', 'Browser_6', 'Browser_7', 'Browser_8', 'Browser_10', 'Browser_13', 'Region_1', 'Region_2', 'Region_3', 'Region_4', 'Region_5',
                        'Region_6', 'Region_7', 'Region_8', 'Region_9', 'TrafficType_1', 'TrafficType_2', 
                        'TrafficType_3', 'TrafficType_4', 'TrafficType_5', 'TrafficType_6', 'TrafficType_7', 
                        'TrafficType_8', 'TrafficType_9', 'TrafficType_10', 'TrafficType_11', 'TrafficType_13', 
                        'TrafficType_15', 'TrafficType_19', 'TrafficType_20', 'VisitorType_New_Visitor', 'VisitorType_Other', 'VisitorType_Returning_Visitor', 'Weekend_False', 'Weekend_True']
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(X_train_full[numeric_features]))
test_scaled = pd.DataFrame(scaler.fit_transform(X_test_full[numeric_features]))
train_scaled.columns = X_train_full[numeric_features].columns
test_scaled.columns = X_test_full[numeric_features].columns
X_train_scaled = pd.concat([train_scaled,X_train_full[categorical_features]],axis=1)
X_test_scaled = pd.concat([test_scaled,X_test_full[categorical_features]],axis=1)

In [13]:
from sklearn.feature_selection import VarianceThreshold

threshold = 0.0075  # Soglia di varianza desiderata
variance_selector = VarianceThreshold(threshold=threshold)

# Applicazione della selezione della varianza sul dataset

X_train_scaled = X_train_scaled.loc[:, variance_selector.fit(X_train_scaled).get_support()]
columns_remaining = X_train_scaled.columns.tolist()

X_test_scaled = X_test_scaled[columns_remaining]
print(X_train_scaled.columns.tolist())
len(columns_remaining)

['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jul', 'Month_June', 'Month_Mar', 'Month_May', 'Month_Nov', 'Month_Oct', 'Month_Sep', 'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3', 'OperatingSystems_4', 'Browser_1', 'Browser_2', 'Browser_3', 'Browser_4', 'Browser_5', 'Browser_6', 'Browser_8', 'Browser_10', 'Region_1', 'Region_2', 'Region_3', 'Region_4', 'Region_5', 'Region_6', 'Region_7', 'Region_8', 'Region_9', 'TrafficType_1', 'TrafficType_2', 'TrafficType_3', 'TrafficType_4', 'TrafficType_5', 'TrafficType_6', 'TrafficType_8', 'TrafficType_10', 'TrafficType_11', 'TrafficType_13', 'TrafficType_20', 'VisitorType_New_Visitor', 'VisitorType_Returning_Visitor', 'Weekend_False', 'Weekend_True']


56

### Functions and algorithms

#### Feature selection: Forward stepwise selection
We chose this algorithm because it's more scalable on a big dataset.

In [17]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def get_evaluator(scorer):
    def evaluator(model, X, y, trained=False):
        if not trained:
            model = model.fit(X, y)
        score = scorer(model, X, y)
        return model, score
    return evaluator   

def get_cv_evaluator(scorer, cv=3):
    def evaluator(model, X, y, trained=False):            
        scores = cross_val_score(model, X, y, scoring=scorer, cv=cv)
        if not trained:
            model = model.fit(X, y)
        return model, np.mean(scores)
    
    return evaluator

# Definition of helper functions to generalize forward stepwise selection for different algorithms
def get_logistic_model(solver, penalty, max_iter=1000):
    def logistic_model():
        model = LogisticRegression(solver=solver, penalty=penalty,max_iter=max_iter)
        return model
    return logistic_model

def get_LDA_model():
    def LDA_model():
        model = LDA()
        return model
    return LDA_model

def get_QDA_model():
    def QDA_model():
        model = QDA()
        return model
    return QDA_model

def get_linear_regression_model(fit_intercept=True):
    def linear_regression_model():
        model = LinearRegression(fit_intercept=fit_intercept)
        return model
    return linear_regression_model

def get_SVM_model(kernel,C,max_iter=1000):
    def SVM_model():
        if kernel == 'linear':
            model = LinearSVC(C=C,max_iter=max_iter)
        else:
            model = SCV(kernel=kernel,C=C,max_iter=max_iter)
        return model
    return SVM_model

possible_models = {
    "LogisticRegression":LogisticRegression(solver="saga", penalty='l2',max_iter=1000),
    "LDA":LDA(),
    "QDA":QDA(),
    #"KNN":KNeighborsClassifier(n_neighbors=6),
    "LinearRegression":LinearRegression(fit_intercept=True),
    "SVM":LinearSVC(C=0.1,max_iter=1000)
}

#da qui inizia il ctrl Z

def forward_selection(Xtrain_pd, ytrain, model_chosen,
                      candidates_evaluator, candidates_argbest, # Metric to be used at 2.b
                      subsets_evaluator, subsets_argbest,       # Metric to be used at 3
                      candidates_scorer_name=None,  # Name of 2. figure
                      subsets_scorer_name=None,     # Name of 3. figure
                      verbose=True, weight_step3=0):   
    
    
    # Global variable init
    # ====================
    num_features = Xtrain_pd.shape[-1]
    best_candidate_metric = []
    # subsets_* are lists containing one value for each Mk model (the best of the Mk candidates)
    subsets_metric = []        # The best metric of each subset of dimension 'dim'
    subsets_best_features = [] # The best features combination in each subset of dimension 'dim'
    # A figure to keep track of candidates scores in each Mk subset
    num_evaluations = 0        # A conter to keep track of the total number of trials
    
    selected_features = []
    all_features = Xtrain_pd.columns
    
    
    # 1. Train M0
    # ===========
    model = DummyRegressor()
    # Compute (2.b) metrics
    model, score = candidates_evaluator(model, Xtrain_pd[[]], ytrain)
    best_candidate_metric.append(score)
    subsets_best_features.append([])
    # Compute metric for step 3.
    _, score = subsets_evaluator(model, Xtrain_pd[[]], ytrain, trained=True)
    subsets_metric.append(score)
    
    
    for dim in range(num_features):
        candidate_metrics = [] # metrics for all the models with dim features
        candidate_models = []  # models with dim features
        
        remaining_features = all_features.difference(selected_features)
        
        # fit all the models with k features
        for new_column in remaining_features:
            Xtrain_sub = Xtrain_pd[selected_features+[new_column]].to_numpy()
            #model = possible_models[model_chosen]
            model = model_chosen()
            model, score = candidates_evaluator(model, Xtrain_sub, ytrain)
            candidate_models.append(model)
            candidate_metrics.append(score)
            num_evaluations += 1
            
        
        idx_best_candidate = candidates_argbest(candidate_metrics) # select the best Mk model
        selected_features.append(remaining_features[idx_best_candidate]) # Update selected feature
        best_candidate_metric.append(candidate_metrics[idx_best_candidate]) # Save best candidate features
        best_features = selected_features.copy()
        subsets_best_features.append(best_features)
        
        
        # Compute metric for the final step -> comparison of all the best models
        best_subset_model = candidate_models[idx_best_candidate] # save the best model
        best_subset_Xtrain = Xtrain_pd[best_features].to_numpy()
        _, score = subsets_evaluator(best_subset_model, best_subset_Xtrain, ytrain, trained=True)
        subsets_metric.append(score) #computing the metrics for the training set
        num_evaluations += weight_step3 
        
        if verbose:
            print("............")
            print("Best model (M{}) with {} features: {}".format(dim+1, dim+1, best_features))
            print("M{} subset score (3.): {}".format(dim+1, score))
        
    # choose the best candidates
    best_subset_idx = subsets_argbest(subsets_metric)
    best_features = subsets_best_features[best_subset_idx]
    
    if verbose:
        print("\n Best configuration has {} features".format(best_subset_idx))
        print("Features: {}".format(subsets_best_features[best_subset_idx]))
        print("Total number of trained models:", num_evaluations)
    
    # Complete the subsets_fig figure by plotting
    # a line connecting all best candidate score
    best_candidate_score_idx = candidates_argbest(best_candidate_metric)
    return best_features
    

#### Accuracy

In [7]:
def accuracy(y_pred, y_true):
    return (y_pred == y_true).mean()

def calculate_sensitivity_specificity(confusion_matrix):
    # Extract values from the confusion matrix
    TN, FP, FN, TP = confusion_matrix.ravel()

    # Calculate Sensitivity (Recall)
    sensitivity = TP / (TP + FN)

    # Calculate Specificity
    specificity = TN / (TN + FP)

    return sensitivity, specificity

#### Evaluation metrics

In [8]:
from sklearn.metrics import confusion_matrix

def classification_metrics_training(model, model_name, X_train, y_train, cv):
    #sensitivity
    sensitivity_train = np.mean(np.absolute(cross_val_score(model, X_train, y_train, cv=cv, scoring='recall', n_jobs=-1)))
    
    #specificity
    specificity_train = np.mean(np.absolute(cross_val_score(model, X_train, y_train, cv=cv, scoring='precision', n_jobs=-1)))
    
    #accuracy
    accuracy_train = np.mean(np.absolute(cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)))
                         
    # building a dataframe
    data = {
        'Model': [model_name],
        'Accuracy': [accuracy_train],
        'Sensitivity': [sensitivity_train],
        'Specificity': [specificity_train],
    }
    return pd.DataFrame(data)                    

In [9]:
def classification_metrics_test(model, model_name,X_train,y_train,X_test,y_test,threshold):
    # building the confusion matrix for test performances
    model = model.fit(X_train,y_train)
    #ypred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability of positive class (class 1)

    # Convert probabilities to binary predictions
    ypred = (y_pred_proba > threshold).astype(int)
    
    cm = confusion_matrix(y_test.to_numpy(), np.array(ypred))
    sensitivity_test, specificity_test = calculate_sensitivity_specificity(cm)
    
    accuracy_test = accuracy(ypred, y_test)
    
    # building a dataframe
    data = {
        'Model': [model_name],
        'Accuracy': [accuracy_test],
        'Sensitivity': [sensitivity_test],
        'Specificity': [specificity_test],
    }
    return pd.DataFrame(data), cm

In [10]:
# We compute the empirical threshold
empirical_threshold = y_train.mean()

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def classification_metrics_empirical_threshold(model, model_name, X_train, y_train,threshold=0.5):
    
    model = model.fit(X_train, y_train)
    
    # Get predicted probabilities
    y_pred_proba = model.predict_proba(X_train)[:, 1]  # Probability of positive class (class 1)

    # Convert probabilities to binary predictions
    y_pred = (y_pred_proba > threshold).astype(int)

    # Evaluate the performance using various metrics
    accuracy = accuracy_score(y_train, y_pred)
    specificity = precision_score(y_train, y_pred)
    sensitivity = recall_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    
    data = {
        'Model': [model_name],
        'Accuracy': [accuracy],
        'Sensitivity': [sensitivity],
        'Specificity': [specificity],
    }
    return pd.DataFrame(data) 


## Logistic regression

### Scaling data
We scale the numerical features to avoid convergence problem with logistic regression.

In [12]:
# FACCIAMO QUESTO STEP PER EVITARE PROBLEMI DI CONVERGENZA NEL MODELLO
from sklearn.preprocessing import StandardScaler

# Assuming you have your feature data X
numeric_features = ['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']
categorical_features = ['Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jul', 'Month_June', 'Month_Mar', 'Month_May', 
                        'Month_Nov', 'Month_Oct', 'Month_Sep', 'OperatingSystems_1', 'OperatingSystems_2', 'OperatingSystems_3', 'OperatingSystems_4', 'OperatingSystems_6', 'OperatingSystems_8', 'Browser_1', 'Browser_2', 'Browser_3', 'Browser_4', 'Browser_5', 'Browser_6', 'Browser_7', 'Browser_8', 'Browser_10', 'Browser_13', 'Region_1', 'Region_2', 'Region_3', 'Region_4', 'Region_5',
                        'Region_6', 'Region_7', 'Region_8', 'Region_9', 'TrafficType_1', 'TrafficType_2', 
                        'TrafficType_3', 'TrafficType_4', 'TrafficType_5', 'TrafficType_6', 'TrafficType_7', 
                        'TrafficType_8', 'TrafficType_9', 'TrafficType_10', 'TrafficType_11', 'TrafficType_13', 
                        'TrafficType_15', 'TrafficType_19', 'TrafficType_20', 'VisitorType_New_Visitor', 'VisitorType_Other', 'VisitorType_Returning_Visitor', 'Weekend_False', 'Weekend_True']
scaler = StandardScaler()
train_scaled = pd.DataFrame(scaler.fit_transform(X_train[numeric_features]))
test_scaled = pd.DataFrame(scaler.fit_transform(X_test[numeric_features]))
train_scaled.columns = X_train[numeric_features].columns
test_scaled.columns = X_test[numeric_features].columns
X_train_scaled = pd.concat([train_scaled,X_train[categorical_features]],axis=1)
X_test_scaled = pd.concat([test_scaled,X_test[categorical_features]],axis=1)

### Feature selection

In [16]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
features_logistic = forward_selection(X_train_scaled, y_train, get_logistic_model('newton-cg', 'none', max_iter=1000),
                  get_evaluator(make_scorer(accuracy)), np.argmax, # 2.
                  get_cv_evaluator(make_scorer(accuracy), cv), np.argmax, # 3.
                  candidates_scorer_name="Accuracy",
                  subsets_scorer_name="Accuracy (CV)",
                  verbose=True, weight_step3=cv.n_splits)
features_logistic

............
Best model (M1) with 1 features: ['PageValues']
M1 subset score (3.): 0.8802981889406984
............
Best model (M2) with 2 features: ['PageValues', 'VisitorType_New_Visitor']
M2 subset score (3.): 0.8823527107421105
............
Best model (M3) with 3 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3']
M3 subset score (3.): 0.8830015347960183
............
Best model (M4) with 4 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10']
M4 subset score (3.): 0.8837583500211948
............
Best model (M5) with 5 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Browser_7']
M5 subset score (3.): 0.8835421338049786
............
Best model (M6) with 6 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Browser_7', 'Region_8']
M6 subset score (3.): 0.8836503003815064
............
Best model (M7) with 7 features: ['PageValues', 'VisitorType_New_Visitor', 'Traf

............
Best model (M28) with 28 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Browser_7', 'Region_8', 'TrafficType_11', 'Browser_3', 'Browser_8', 'Month_Feb', 'Browser_1', 'OperatingSystems_8', 'Region_5', 'Month_Aug', 'TrafficType_7', 'TrafficType_9', 'Region_2', 'Browser_13', 'OperatingSystems_6', 'Browser_4', 'VisitorType_Other', 'Browser_5', 'Region_6', 'TrafficType_15', 'Month_June', 'TrafficType_19', 'VisitorType_Returning_Visitor', 'Browser_10']
M28 subset score (3.): 0.8840829082191982
............
Best model (M29) with 29 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Browser_7', 'Region_8', 'TrafficType_11', 'Browser_3', 'Browser_8', 'Month_Feb', 'Browser_1', 'OperatingSystems_8', 'Region_5', 'Month_Aug', 'TrafficType_7', 'TrafficType_9', 'Region_2', 'Browser_13', 'OperatingSystems_6', 'Browser_4', 'VisitorType_Other', 'Browser_5', 'Region_6', 'TrafficType_15', 'Month_June', 'TrafficType_

............
Best model (M41) with 41 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Browser_7', 'Region_8', 'TrafficType_11', 'Browser_3', 'Browser_8', 'Month_Feb', 'Browser_1', 'OperatingSystems_8', 'Region_5', 'Month_Aug', 'TrafficType_7', 'TrafficType_9', 'Region_2', 'Browser_13', 'OperatingSystems_6', 'Browser_4', 'VisitorType_Other', 'Browser_5', 'Region_6', 'TrafficType_15', 'Month_June', 'TrafficType_19', 'VisitorType_Returning_Visitor', 'Browser_10', 'Month_Jul', 'Region_3', 'Region_7', 'TrafficType_20', 'OperatingSystems_4', 'OperatingSystems_1', 'TrafficType_4', 'TrafficType_5', 'Month_Oct', 'TrafficType_8', 'BounceRates', 'Month_Dec', 'Month_Mar']
M41 subset score (3.): 0.8837584084896146
............
Best model (M42) with 42 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Browser_7', 'Region_8', 'TrafficType_11', 'Browser_3', 'Browser_8', 'Month_Feb', 'Browser_1', 'OperatingSystems_8', 'Region

............
Best model (M51) with 51 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Browser_7', 'Region_8', 'TrafficType_11', 'Browser_3', 'Browser_8', 'Month_Feb', 'Browser_1', 'OperatingSystems_8', 'Region_5', 'Month_Aug', 'TrafficType_7', 'TrafficType_9', 'Region_2', 'Browser_13', 'OperatingSystems_6', 'Browser_4', 'VisitorType_Other', 'Browser_5', 'Region_6', 'TrafficType_15', 'Month_June', 'TrafficType_19', 'VisitorType_Returning_Visitor', 'Browser_10', 'Month_Jul', 'Region_3', 'Region_7', 'TrafficType_20', 'OperatingSystems_4', 'OperatingSystems_1', 'TrafficType_4', 'TrafficType_5', 'Month_Oct', 'TrafficType_8', 'BounceRates', 'Month_Dec', 'Month_Mar', 'Region_1', 'TrafficType_1', 'Month_Sep', 'OperatingSystems_2', 'OperatingSystems_3', 'Browser_2', 'Browser_6', 'Region_4', 'Region_9', 'TrafficType_2']
M51 subset score (3.): 0.8814876120035666
............
Best model (M52) with 52 features: ['PageValues', 'VisitorType_New_Visitor', 'Traff

............
Best model (M60) with 60 features: ['PageValues', 'VisitorType_New_Visitor', 'TrafficType_3', 'TrafficType_10', 'Browser_7', 'Region_8', 'TrafficType_11', 'Browser_3', 'Browser_8', 'Month_Feb', 'Browser_1', 'OperatingSystems_8', 'Region_5', 'Month_Aug', 'TrafficType_7', 'TrafficType_9', 'Region_2', 'Browser_13', 'OperatingSystems_6', 'Browser_4', 'VisitorType_Other', 'Browser_5', 'Region_6', 'TrafficType_15', 'Month_June', 'TrafficType_19', 'VisitorType_Returning_Visitor', 'Browser_10', 'Month_Jul', 'Region_3', 'Region_7', 'TrafficType_20', 'OperatingSystems_4', 'OperatingSystems_1', 'TrafficType_4', 'TrafficType_5', 'Month_Oct', 'TrafficType_8', 'BounceRates', 'Month_Dec', 'Month_Mar', 'Region_1', 'TrafficType_1', 'Month_Sep', 'OperatingSystems_2', 'OperatingSystems_3', 'Browser_2', 'Browser_6', 'Region_4', 'Region_9', 'TrafficType_2', 'ExitRates', 'TrafficType_6', 'TrafficType_13', 'Informational_Duration', 'Weekend_False', 'Weekend_True', 'Administrative_Duration', 'Spe

['PageValues',
 'VisitorType_New_Visitor',
 'TrafficType_3',
 'TrafficType_10',
 'Browser_7',
 'Region_8',
 'TrafficType_11',
 'Browser_3',
 'Browser_8',
 'Month_Feb']

### Evaluation Metrics

In [ ]:
model_logistic = LogisticRegression(solver="newton-cg",penalty="none",max_iter=1000)
train_scores_logistic = classification_metrics_training(model_logistic, "LogisticRegression", X_train_scaled[features_logistic], y_train,cv)
train_scores_logistic

The sensitivity is very low, we try to improve it using the empirical threshold

In [ ]:
train_scores_logistic_empirical = classification_metrics_empirical_threshold(model_logistic, 'LogisticRegression', X_train_scaled[features_logistic], y_train,empirical_threshold)
train_scores_logistic_empirical

# LDA

### Feature selection

In [18]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)
features_LDA = forward_selection(X_train_scaled, y_train, get_LDA_model(),
                  get_evaluator(make_scorer(accuracy)), np.argmax, # 2.
                  get_cv_evaluator(make_scorer(accuracy), cv), np.argmax, # 3.
                  candidates_scorer_name="Accuracy",
                  subsets_scorer_name="Accuracy (CV)",
                  verbose=True, weight_step3=cv.n_splits)
features_LDA

............
Best model (M1) with 1 features: ['PageValues']
M1 subset score (3.): 0.8766217531755661
............
Best model (M2) with 2 features: ['PageValues', 'ExitRates']
M2 subset score (3.): 0.8783517167789748
............
Best model (M3) with 3 features: ['PageValues', 'ExitRates', 'TrafficType_3']
M3 subset score (3.): 0.8782437256077061
............
Best model (M4) with 4 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2']
M4 subset score (3.): 0.8785680499320303
............
Best model (M5) with 5 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2']
M5 subset score (3.): 0.8792168739859385
............
Best model (M6) with 6 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates']
M6 subset score (3.): 0.8794332071389942
............
Best model (M7) with 7 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRat

............
Best model (M28) with 28 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'VisitorType_Other', 'Month_Sep', 'Browser_10', 'Browser_13', 'Browser_3', 'Browser_7', 'Browser_8', 'Month_Feb', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Month_Aug', 'OperatingSystems_4', 'OperatingSystems_6', 'OperatingSystems_8', 'Region_7', 'Region_2', 'Region_8', 'TrafficType_15', 'TrafficType_19']
M28 subset score (3.): 0.8800819727244822
............
Best model (M29) with 29 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'VisitorType_Other', 'Month_Sep', 'Browser_10', 'Browser_13', 'Browser_3', 'Browser_7', 'Browser_8', 'Month_Feb', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Month_Aug', 'OperatingSystems_4', 'OperatingSystems_6', 'OperatingSystems_8', 'Region_7', 'Region_2', 'Region_8', 'TrafficType_1

............
Best model (M42) with 42 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'VisitorType_Other', 'Month_Sep', 'Browser_10', 'Browser_13', 'Browser_3', 'Browser_7', 'Browser_8', 'Month_Feb', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Month_Aug', 'OperatingSystems_4', 'OperatingSystems_6', 'OperatingSystems_8', 'Region_7', 'Region_2', 'Region_8', 'TrafficType_15', 'TrafficType_19', 'TrafficType_6', 'TrafficType_7', 'TrafficType_9', 'Month_June', 'Month_Oct', 'Region_5', 'Browser_2', 'Browser_1', 'Region_4', 'Browser_4', 'Browser_6', 'Region_1', 'OperatingSystems_1', 'OperatingSystems_3']
M42 subset score (3.): 0.8798658734451056
............
Best model (M43) with 43 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'VisitorType_Other', 'Month_Sep', 'Browser_10', 'Browser_13', 'Browser_3', 'Browser_7'

............
Best model (M53) with 53 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'VisitorType_Other', 'Month_Sep', 'Browser_10', 'Browser_13', 'Browser_3', 'Browser_7', 'Browser_8', 'Month_Feb', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Month_Aug', 'OperatingSystems_4', 'OperatingSystems_6', 'OperatingSystems_8', 'Region_7', 'Region_2', 'Region_8', 'TrafficType_15', 'TrafficType_19', 'TrafficType_6', 'TrafficType_7', 'TrafficType_9', 'Month_June', 'Month_Oct', 'Region_5', 'Browser_2', 'Browser_1', 'Region_4', 'Browser_4', 'Browser_6', 'Region_1', 'OperatingSystems_1', 'OperatingSystems_3', 'Region_6', 'Region_9', 'TrafficType_20', 'Month_Jul', 'TrafficType_4', 'TrafficType_10', 'SpecialDay', 'Administrative_Duration', 'VisitorType_New_Visitor', 'Month_Dec', 'ProductRelated_Duration']
M53 subset score (3.): 0.8786766842559164
............
Best model (M54) with 54 features: ['PageValues

............
Best model (M62) with 62 features: ['PageValues', 'ExitRates', 'TrafficType_3', 'OperatingSystems_2', 'TrafficType_2', 'BounceRates', 'Region_3', 'Browser_5', 'VisitorType_Other', 'Month_Sep', 'Browser_10', 'Browser_13', 'Browser_3', 'Browser_7', 'Browser_8', 'Month_Feb', 'TrafficType_13', 'TrafficType_11', 'TrafficType_5', 'Month_Aug', 'OperatingSystems_4', 'OperatingSystems_6', 'OperatingSystems_8', 'Region_7', 'Region_2', 'Region_8', 'TrafficType_15', 'TrafficType_19', 'TrafficType_6', 'TrafficType_7', 'TrafficType_9', 'Month_June', 'Month_Oct', 'Region_5', 'Browser_2', 'Browser_1', 'Region_4', 'Browser_4', 'Browser_6', 'Region_1', 'OperatingSystems_1', 'OperatingSystems_3', 'Region_6', 'Region_9', 'TrafficType_20', 'Month_Jul', 'TrafficType_4', 'TrafficType_10', 'SpecialDay', 'Administrative_Duration', 'VisitorType_New_Visitor', 'Month_Dec', 'ProductRelated_Duration', 'Month_Mar', 'VisitorType_Returning_Visitor', 'Weekend_False', 'Informational_Duration', 'Weekend_True

['PageValues',
 'ExitRates',
 'TrafficType_3',
 'OperatingSystems_2',
 'TrafficType_2',
 'BounceRates',
 'Region_3',
 'Browser_5',
 'VisitorType_Other',
 'Month_Sep',
 'Browser_10',
 'Browser_13',
 'Browser_3',
 'Browser_7',
 'Browser_8',
 'Month_Feb',
 'TrafficType_13',
 'TrafficType_11',
 'TrafficType_5',
 'Month_Aug',
 'OperatingSystems_4',
 'OperatingSystems_6',
 'OperatingSystems_8',
 'Region_7']

### Evaluation metrics

In [ ]:
model_LDA =LDA(store_covariance=True)
train_scores_LDA = classification_metrics_training(model_LDA, "LDA", X_train_scaled[features_LDA], y_train, cv)
train_scores_LDA

We try to improve the sensitivity by setting the empirical threshold

In [ ]:
train_scores_LDA_empirical = classification_metrics_empirical_threshold(model_LDA, 'LDA', X_train_scaled[features_LDA], y_train,empirical_threshold)
train_scores_LDA_empirical

## QDA

### Feature selection

In [19]:
#import warnings
#warnings.filterwarnings("ignore")
cv = KFold(n_splits=5, random_state=42, shuffle=True)
features_QDA = forward_selection(X_train_scaled, y_train, get_QDA_model(),
                  get_evaluator(make_scorer(accuracy)), np.argmax, # 2.
                  get_cv_evaluator(make_scorer(accuracy), cv), np.argmax, # 3.
                  candidates_scorer_name="Accuracy",
                  subsets_scorer_name="Accuracy (CV)",
                  verbose=False, weight_step3=cv.n_splits)
features_QDA

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:906: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:907: RuntimeWarning: invalid value encountered in add
  return -0.5 * (norm2 + u) + np.log(self.priors_)
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: divide by zero encountered in power
 

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:906: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variable

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:903: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:906: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variable

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-

...

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\sarar\anaconda3\lib\site-

['PageValues', 'BounceRates', 'VisitorType_New_Visitor']

### Evaluation

In [ ]:
model_QDA = QDA(store_covariance=True)
train_scores_QDA = classification_metrics_training(model_QDA, "QDA", X_train_scaled[features_QDA], y_train, cv)
train_scores_QDA

The sensitivity is very high with respect to the specificity, so we try to set the empirical threshold

In [ ]:
train_scores_QDA_empirical = classification_metrics_empirical_threshold(model_QDA, 'QDA', X_train_scaled[features_QDA], y_train, empirical_threshold)
train_scores_QDA_empirical

## KNN

### With all features

In [ ]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

In [ ]:
model_KNN_all = KNeighborsClassifier()
params = {'n_neighbors': range(1, 15)}
cv = KFold(n_splits=5, random_state=42, shuffle=True)
cv_KNN_all = GridSearchCV(model_KNN_all, params, refit=True, cv=cv, 
                  scoring=make_scorer(accuracy))
cv_KNN_all.fit(X_train, y_train)

In [ ]:
model_KNN_all = KNeighborsClassifier(n_neighbors=cv_KNN_all.best_params_['n_neighbors'])
train_scores_KNN_all = classification_metrics_training(model_KNN_all, "KNN", X_train, y_train, cv)
train_scores_KNN_all

In [ ]:
train_scores_KNN_all_empirical = classification_metrics_empirical_threshold(model_KNN_all, 'KNN', X_train, y_train,empirical_threshold)
train_scores_KNN_all_empirical

### With only numeric features
KNN has two main issues: 
1. It is not scalable with a large amount of features
2. It relies on a distance metric to compute the distance between points. This distance may not be significant in case of categorical variables (even if we use one-hot encoding)
3. From the previous analysis we can also notice that even with the empirical threshold the performances don't improve significantly

So for these reasons we also fit a model with just the numeric features.

In [ ]:
model_KNN_numeric = KNeighborsClassifier()
params = {'n_neighbors': range(1, 50)}
cv_KNN_numeric = GridSearchCV(model_KNN_numeric, params, refit=True, cv=cv, 
                  scoring=make_scorer(accuracy))
cv_KNN_numeric.fit(X_train[numeric_features], y_train)
model_KNN_numeric = KNeighborsClassifier(n_neighbors=cv_KNN_numeric.best_params_['n_neighbors'])


In [ ]:
train_scores_KNN_numeric = classification_metrics_training(model_KNN_numeric, "KNN", X_train[numeric_features], y_train, cv)
train_scores_KNN_numeric

In [ ]:
train_scores_KNN_numeric_empirical = classification_metrics_empirical_threshold(model_KNN_numeric, 'KNN', X_train[numeric_features], y_train, empirical_threshold)
train_scores_KNN_numeric_empirical

## SVM

### Choosing the optimal kernel

In [14]:
kernels = ['linear', 'rbf', 'poly', 'sigmoid']
score = []
for kern in kernels:
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    print(kern)
    if kern == 'linear':
        print('sono linear...')
        model_SVM = LinearSVC()
    else:
        print('nonlinear')
        model_SVM = SVC(kernel=kern)
    scores = cross_val_score(model_SVM, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    score.append(np.mean(scores))

max_index = score.index(max(score))
best_kernel = kernels[max_index]
acc_best_model = max(score)

linear
sono linear...
rbf
nonlinear
poly
nonlinear
sigmoid
nonlinear


NameError: name 'time' is not defined

### Choosing the best C

In [15]:
C_s= [0.1,1,10,100]

score_c = []
   
for c in C_s:
    model_SVM = LinearSVC(C=c)
    cv = KFold(n_splits=5, random_state=1, shuffle=True)
    scores_score = cross_val_score(model_SVM, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    score_c.append(np.mean(scores_score))
best_score_c = score_c.index(max(score_c))
   

best_c = C_s[best_score_c]
best_c

0.1

In [20]:
import warnings
#warnings.filterwarnings("ignore")
warnings.filterwarnings("default")

cv = KFold(n_splits=5, random_state=42, shuffle=True)
features_SVM = forward_selection(X_train_scaled, y_train, get_SVM_model('linear',0.1,2000),
                  get_evaluator(make_scorer(accuracy)), np.argmax, # 2.
                  get_cv_evaluator(make_scorer(accuracy), cv), np.argmax, # 3.
                  candidates_scorer_name="Accuracy",
                  subsets_scorer_name="Accuracy (CV)",
                  verbose=True, weight_step3=cv.n_splits)
features_SVM

............
Best model (M1) with 1 features: ['PageValues']
M1 subset score (3.): 0.8788924911931943
............
Best model (M2) with 2 features: ['PageValues', 'BounceRates']
M2 subset score (3.): 0.8791087658778303
............
Best model (M3) with 3 features: ['PageValues', 'BounceRates', 'TrafficType_1']
M3 subset score (3.): 0.8795412567786824
............
Best model (M4) with 4 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11']
M4 subset score (3.): 0.8796493648867905
............
Best model (M5) with 5 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Browser_7']
M5 subset score (3.): 0.8796493648867905
............
Best model (M6) with 6 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Browser_7', 'Browser_3']
M6 subset score (3.): 0.8796493648867905
............
Best model (M7) with 7 features: ['PageValues', 'BounceRates', 'TrafficType_1', 'TrafficType_11', 'Browser_7', 'Browser_3', 'VisitorType_Ot

KeyboardInterrupt: 

In [26]:
model_SVM = LinearSVC()
model_SVM = model_SVM.fit(X_train_scaled[features_SVM],y_train)

C:\Users\sarar\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [28]:
train_scores_SVM = classification_metrics_training(model_SVM, "SVM", X_train_scaled[features_SVM], y_train, cv)
train_scores_SVM

,Model,Accuracy,Sensitivity,Specificity
0,SVM,0.881055,0.331061,0.780808


In [30]:
#train_scores_SVM_empirical = classification_metrics_empirical_threshold(model_SVM, 'SVM', X_train_scaled[features_SVM], y_train, empirical_threshold)
#train_scores_SVM_empirical

## Algorithm comparison

### Metric comparison

#### Threshold = 0.5

In [31]:
metrics_df = pd.concat([train_scores_logistic,train_scores_LDA,train_scores_QDA,train_scores_KNN_all,train_scores_KNN_numeric,train_scores_SVM],axis=0)
metrics_df

NameError: name 'train_scores_logistic' is not defined

#### Empirical threshold

In [ ]:
metrics_df_empirical = pd.concat([train_scores_logistic_empirical,train_scores_LDA_empirical,train_scores_QDA_empirical,train_scores_KNN_all_empirical,train_scores_KNN_numeric_empirical],axis=0)
metrics_df_empirical

In [ ]:
from sklearn.metrics import roc_curve

calibrated_svc = CalibratedClassifierCV(model_SVM, method='sigmoid', cv='prefit')

def plot_roc(predict_fn, X, y, label=None):
    fprs, tprs, t = roc_curve(y, predict_fn(X)[:,-1])
    
    # Plot the ROC
    plt.plot(fprs, tprs, label="ROC "+label)
    plt.xlabel("FPR = 1 - specificity")
    plt.ylabel("TPR = sensitivity")
    plt.legend()

auc_list = []

for name, model, X in [('lr', model_logistic, X_train_scaled[features_logistic]), 
                    ('lda', model_LDA, X_train_scaled[features_LDA]), 
                    ('qda', model_QDA, X_train_scaled[features_QDA]),
                    ('SVC', calibrated_svc,X_train_scaled[features_SVM]),
                    ('KNN-all', model_KNN_all, X_train),
                    ('KNN-numeric', model_KNN_numeric, X_train[numeric_features])
                   ]:
    model.fit(X, y_train) # sistemare questa parte, fitta i modelli prima in modo che sia tutto ok
    plot_roc(model.predict_proba, X, y_train, name)
    y_prob_pred = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y_train, y_prob_pred)
    auc_list.append(auc)
auc_list